In [ ]:
import pandas as pd
df = pd.read_csv('papers.csv')

In [ ]:
unique_affiliations_df = df['Affiliation'].unique()
# add first head drop
unique_affiliations_df = pd.DataFrame(unique_affiliations_df, columns=['Affiliation'])
pd.DataFrame(unique_affiliations_df).to_csv('unique_affiliations.csv')

In [ ]:
unique_affiliations_df.head()
import requests
from tqdm import tqdm

unique_affiliations_df['country'] = None

none_count = 0

# first search with openstreetmap
for i in tqdm(range(len(unique_affiliations_df))):
    try:
        institute_name = unique_affiliations_df.loc[i, 'Affiliation']
        url = f"https://nominatim.openstreetmap.org/search?q={institute_name}&format=json&accept-language=en"

        response = requests.get(url)
        if response.status_code == 200:
            results = response.json()
            if len(results) > 0:
                country = results[0]['display_name'].split(',')[-1].strip()
                unique_affiliations_df.loc[i, 'country'] = country
            else:
                unique_affiliations_df['country'][i] = 'None'
                none_count += 1
        else:
            print(f"API ERROR")
    except:
        print(f"ERROR: {i}")
        unique_affiliations_df['country'][i] = 'None'
        none_count += 1
print(f"None count: {none_count}")
pd.DataFrame(unique_affiliations_df).to_csv('unique_affiliations.csv')


In [ ]:
unique_affiliations_df['country'].value_counts()

In [ ]:
import re
import json
from tqdm import tqdm
import requests
import pandas as pd

# separate None in country df
unique_affiliations_df = pd.read_csv('unique_affiliations.csv')
none_df = pd.DataFrame(unique_affiliations_df[unique_affiliations_df['country'] == 'None'].reset_index(drop=True).copy())

none_count = 0

for i in tqdm(range(len(none_df))):
    try:
        institute_name = none_df.loc[i, 'Affiliation']

        # Your Scopus API key
        api_key = "46628ca2f498070f25172d9c78fd1c86"

        # Search query parameters
        query = f"affil({institute_name})"

        # API request URL
        url = f"https://api.elsevier.com/content/search/affiliation?query={query}&apiKey={api_key}"

        # Send request to API
        response = requests.get(url)
        #get json
        data = response.json()
        if len(data['search-results']['entry']) > 0:
            country = data['search-results']['entry'][0]['country']
            none_df.loc[i, 'country'] = country
    except:
        none_count += 1
print(f"None count: {none_count}")
pd.DataFrame(none_df).to_csv('none_df.csv')

In [ ]:
university_affiliation_df = pd.read_csv('unique_affiliations.csv')
none_df = pd.read_csv('none_df.csv')

university_affiliation_dict = {}
for i in range(len(university_affiliation_df)):
    university_affiliation_dict[university_affiliation_df.loc[i, 'Affiliation']] = university_affiliation_df.loc[i, 'country']

for i in range(len(none_df)):
    university_affiliation_dict[none_df.loc[i, 'Affiliation']] = none_df.loc[i, 'country']

university_affiliation_dict

In [17]:
papers = pd.read_csv('papers.csv')
papers['country'] = None
none_count = 0
for i in tqdm(range(len(papers))):
    institute_name = papers.loc[i, 'Affiliation']
    papers.loc[i, 'country'] = university_affiliation_dict[institute_name]

#save papers_with_country
papers.to_csv('papers_with_country.csv')

100%|██████████| 49317/49317 [00:01<00:00, 39740.79it/s]


In [23]:
# read papers_with_country
papers = pd.read_csv('papers_with_country.csv')
#top 

446